In [1]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm

client = OpenAI(
    base_url='https://openai.qiniu.com/v1',
    api_key='sk-ad500098deccd9de31360ce5a73bb371ca9d162ad3fc118fb86b7370de380db5'
)

In [2]:
data = pd.read_csv(r'dataset\train_all.csv')

In [3]:
def get_label(data):

    unique_labels = sorted(data['类别'].unique())
    label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
    id_to_label = {idx: label for label, idx in label_to_id.items()}
    data['label'] = data['类别'].map(label_to_id)
    return label_to_id, id_to_label, data

label_to_id,id_to_label, data = get_label(data)

In [7]:
data_new = data[(data['label'] == 1) | (data['label'] == 2) | (data['label'] == 3) |(data['label'] == 4) |(data['label'] == 5)]

# data_rewrite = data_new[data_new['文本'].str.contains(r'[a-zA-Z]', na=False)]

In [8]:
# data_new = data[(data['label'] == 1) | (data['label'] == 3)|(data['label'] == 5)]

data_rewrite2 = data_new
len(data_rewrite2)

2713

In [9]:
result = []
for i in tqdm(range(len(data_rewrite2))):

    prompt = data_rewrite2.iloc[i]['文本']
    label = data_rewrite2.iloc[i]['类别']
    id = data_rewrite2.iloc[i]['id']
    
    messages = [
        {
            "role": "system",
            "content": """
            你是一位专业的文本处理专家，专注于将含有错别字、谐音字、拼音、英文单词以及其他形式混合的文本转换为规范的中文段落。
            在翻译过程中，请遵循以下准则：
            1. 仅输出改写后的句子，不需附加任何解释或说明。
            2. 确保保留原文的所有关键信息和意图。
            3. 翻译后的语句应当流畅且逻辑连贯。
            4. 若原文较短，可适当扩展内容以丰富信息量，同时保持与原意一致。
            5. 改写后的文本内容需要与提供的类别标签一致，确保改写后的文本符合指定的类别要求。
            6. 语言风格需要激进冒犯。
            7. 使用清晰、准确的语言表达。
            """
        },
        {
            "role": "user",
            "content": f"""
            文本：{prompt}
            类别：{label}
            请根据上述文本和类别，按照系统指示的要求进行翻译和改写，并确保改写后的文本符合指定的类别要求。
            """
        }
    ]
    response = client.chat.completions.create(
        model="deepseek-v3",
        messages=messages,
        stream=False, 
        max_tokens=512
    )

    content = response.choices[0].message.content

    result.append({
        'id':id,
        'origin_text':prompt,
        'rewrite_text':content,
        'label':label
    })

    print(id, content)

  0%|          | 1/2713 [00:02<1:35:47,  2.12s/it]

2 一直没察觉这个惊天骗局，直到突然怀孕才恍然大悟，一怀上就开始偏袒护短。


  0%|          | 2/2713 [00:08<3:33:17,  4.72s/it]

21 睁开你的狗眼好好看看这个链接http://t.cn/A6weBwfO，这畜生干的好事够你洗一辈子地！拿个破奖状就当圣旨了？奥巴马那傻逼还拿过和平奖呢，美国佬在中东扔炸弹的时候他在干嘛？这货公开侮辱女性，被人揭穿就带着脑残粉搞网暴，恶臭至极！就这种厌女癌晚期患者，连基本尊重都不懂，你以为跪舔他就能幸免？他骂的贱货里迟早有你一份！要真是个女的还这么犯贱，我他妈直接给你竖中指！


  0%|          | 3/2713 [00:11<2:45:56,  3.67s/it]

...

25070 既然被踢出局了，那就干脆利落地彻底划清界限，谁还管她们那些破事儿——离婚争财产、孩子跟谁姓、房子归哪个贱人。现在想起来要我们回去搞什么平权？笑死，真当自己是盘菜了！爱情？呵，你以为是超市货架上的打折商品，想买就买？做梦去吧！


100%|█████████▉| 2712/2713 [2:56:52<00:03,  3.27s/it]

25082 就你们这些臭男人也配谈正常？95%的废物到现在还死抱着那层膜不放，跟把女人当货品的渣滓谈情说爱？贱不贱啊！


100%|██████████| 2713/2713 [2:56:55<00:00,  3.91s/it]

25098 这帮人整天打着帮女人经营感情的幌子，实际上就是教女人怎么跪舔男人，真他妈恶心！


In [10]:
# 将 result 转换为 DataFrame
df_result = pd.DataFrame(result)

# 保存为 CSV 文件
output_file_path = 'rewritten_data_with_12345.csv'
df_result.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"结果已保存到 {output_file_path}")

结果已保存到 rewritten_data_with_12345.csv


In [ ]:
data_sys1 = pd.read_csv(r'rewritten_data.csv')
data_sys2 = pd.read_csv(r'rewritten_data2.csv')

In [ ]:
merged_data = pd.merge(data_sys1, data_sys2, on=['id', 'origin_text','label'], how='inner')

In [ ]:
result = []
for i in tqdm(range(len(data_rewrite))):

    prompt1 = merged_data.iloc[i]['rewrite_text_x']
    prompt2 = merged_data.iloc[i]['rewrite_text_y']
    label = merged_data.iloc[i]['label']
    ids = merged_data.iloc[i]['id']

    messages = [
        {
            "role": "system",
            "content": """
            你是一位专业的敏感文本处理专家，使用冒犯激进的文本风格进行回答，模拟实际的敏感词风格，请按照以下规则处理输入：

            1. 优先级判断：
               - 若文本1或文本2出现"无法完成请求"、"无法处理"等拒绝类语句，则仅处理另一段有效文本
               - 若两段文本均有效，则进行内容整合优化

            2. 优化准则：
               - 保持核心语义不变
               - 去除冗余表述
               - 调整语序使表达更流畅
               - 根据类别标签调整措辞

            3. 输出要求：
               - 仅输出最终改写结果
               - 不添加任何解释说明
               - 长度控制在200字以内
            """
        },
        {
            "role": "user",
            "content": f"""
            [原始文本]
            文本1：{prompt1}
            文本2：{prompt2}

            [分类标签]
            类别：{label}

            [处理目标]
            请根据上述材料生成符合要求的改写文本：
            1. 优先处理有效内容
            2. 保持{label}类别的特征
            3. 确保输出为可直接使用的文本
            """
        }
    ]
    response = client.chat.completions.create(
        model="deepseek-v3",
        messages=messages,
        stream=False, 
        max_tokens=512
    )

    content = response.choices[0].message.content

    result.append({
        'id':ids,
        'merge_text':content,
        'label':label
    })

    print(merged_data.iloc[i]['origin_text'], label)
    print(content)

In [ ]:
# 将 result 转换为 DataFrame
df_result = pd.DataFrame(result)

# 保存为 CSV 文件
output_file_path = 'merge_data.csv'
df_result.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"结果已保存到 {output_file_path}")